# Imports & Settings


In [1]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [4]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

/tmp/ipykernel_9627/1699356385.py:9: DtypeWarning: Columns (13,266) have mixed types. Specify dtype option on import or set low_memory=False.
  prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})


# Preparation

In [106]:
data = prep.copy()

In [42]:
text_features = ['name', 'commentary', 'text', 'legend', 'initial_info', 'additional_text']

In [107]:
data[text_features] = data[text_features].fillna('')

In [91]:
def collect_text(item):
    return ' '.join(item[text_features]).strip()

In [109]:
data['text_features'] = data.apply(lambda item: collect_text(item),axis=1)

In [138]:
data[['name', 'commentary', 'text', 'legend', 'initial_info', 'additional_text', 'text_features']].head(2)

,name,commentary,text,legend,initial_info,additional_text,text_features
id,,,,,,,
232170,"Kuno Areng, Bremerhaven Festwoche medal",NaN,NaN,NaN,Festwoche - Breemenhaven,KUTTER ASTARTE -SCHIFFERGILDE BREMENHAVEN E.V.,"Kuno Areng, Bremerhaven Festwoche medal Festwoche - Breemenhaven KUTTER ASTARTE -SCHIFFERGILDE BREMENHAVEN E.V."
2251378,"Photo-Villem Kapp, photo with dedication to Armilde M, 1937",NaN,NaN,"Photos from the collection of Villem Kapi and Juhan Aavik\ndesse, purchased in 2013",NaN,NaN,"Photo-Villem Kapp, photo with dedication to Armilde M, 1937 Photos from the collection of Villem Kapi and Juhan Aavik\ndesse, purchased in 2013"


In [111]:
data.to_csv('data/prep.csv')

In [ ]:
data = prep.copy()
with_damages = combined_data_fully_translated.copy()

In [ ]:
data.text_features = data.text_features.replace(float('nan'), ' ',)
with_damages.damages = with_damages.damages.replace(float('nan'), ' ',)

data.text_features = data.text_features + ' ' + with_damages.damages

In [ ]:
data.to_csv('data/prep.csv')

In [ ]:
data = prep.copy()

In [ ]:
text = data[['text_features','type']]

In [ ]:
text.to_csv('data/text.csv')

# check for type contained in texts

In [4]:
data = prep.copy()

In [123]:
prep.text_features.info()

<class 'pandas.core.series.Series'>
Int64Index: 20000 entries, 232170 to 2781747
Series name: text_features
Non-Null Count  Dtype 
--------------  ----- 
14900 non-null  object
dtypes: object(1)
memory usage: 312.5+ KB


In [5]:
types = data.type.unique()
# remove nan
types = types[:-1]
types

array(['medal', 'photo', 'photo negative', 'a letter', 'coin',
       'label/sign', 'script, song/vocal music', 'sheet music', 'slide',
       'archaeological find', 'book', 'plan', 'postcard', 'there/imprint',
       'poster', 'design/drawing/sketch', 'audio recording', 'manuscript',
       'document', 'caricature', 'graphics', 'telegram',
       'manuscript, musical composition', 'notes', 'in small print',
       'invitation', 'packaging', 'paper', 'manuscript, sheet music',
       'magazines', 'album', 'in the country', 'digital image',
       'photo, postcard', 'photographic material', 'drawing', 'crate/box',
       'letter of honor/honorary address', 'calendar', 'tape/ribbon',
       'bag', 'newspaper', 'dish/vessel', 'musical instrument',
       'letter, postcard', 'sculpture', 'printed notes', 'there',
       'a suit', 'certificate', 'sheet/linen', 'folder/booklet', 'jewel',
       'doll', 'photographic negative, photographic negative'],
      dtype=object)

In [136]:
data.text_features = data.text_features.replace(float('nan'), '')

In [163]:
multi_counter = 0
one_counter = 0
true_counter = 0
for i, item in data.iterrows():
    if item.source == 'train':
        local_counter = 0
        pred = []
        for type in types:            
            if type in item.text_features:
                local_counter += 1
                pred.append(type)                

        if local_counter > 1:
            multi_counter += 1
        if local_counter == 1:
            one_counter += 1
            if pred[0] == item.type:           
                true_counter += 1
            else:
                #print(pred, item.type) 
                pass

print('multiple types in text - one type in text - one type and true type in text')
print(multi_counter, one_counter, true_counter)
# from 14900 texts

multiple types in text - one type in text - one type and true type in text
483 1642 954
